In [75]:
import pandas as pd

In [76]:
res = pd.read_csv('Dataset/result.csv')

In [77]:
races = res.race.unique()
races

array(['AfricanAmerican', 'Caucasian', 'Other', 'Hispanic', 'Unknown',
       'Asian'], dtype=object)

### Demographic Parity

In [78]:
def demo_parity(race: str):
    num_race = len(res[res['race'] == race])
    predict_true = len(res[(res['predicted'] == 1) & (res['race'] == race)])
    return predict_true/num_race

In [79]:
demo_parity_dict = {}
for race in races:
    demo_parity_dict[race] = demo_parity(race)

In [80]:
demo_parity_dict

{'AfricanAmerican': 0.3929016189290162,
 'Caucasian': 0.38069063461589037,
 'Other': 0.3061760840998686,
 'Hispanic': 0.34960159362549803,
 'Unknown': 0.2237403928266439,
 'Asian': 0.26573426573426573}

### Equalized Opportunity

In [81]:
def equal_opportunity(race: str):
    tp = len(res[(res['readmit_30_days'] == 1) & (res['predicted'] == 1) & (res['race'] == race)] )
    num_race = len(res[res['readmit_30_days'] == 1 & (res['race'] == race) ])
    return tp/num_race

In [82]:
equal_opp_dict = {}
for race in races:
    equal_opp_dict[race] = equal_opportunity(race)

equal_opp_dict
    

{'AfricanAmerican': 0.015799379688784032,
 'Caucasian': 0.1444486643878972,
 'Other': 0.0007389493483810291,
 'Hispanic': 0.0011021886317115414,
 'Unknown': 0.0008129346942462289,
 'Asian': 0.0003329412025836237}

### Equalized Odds

In [83]:
def equalized_odds(race: str):
    tp = len(res[(res['readmit_30_days'] == 1) & (res['predicted'] == 1) & (res['race'] == race)] )
    positive_num_race = len(res[res['readmit_30_days'] == 1 & (res['race'] == race) ])
    fp = len(res[(res['readmit_30_days'] == 0) & (res['predicted'] == 1) & (res['race'] == race)] )
    negative_num_race = len(res[res['readmit_30_days'] == 0 & (res['race'] == race) ])
    return tp/positive_num_race, fp/negative_num_race

In [84]:
equal_odd_dict = {}
for race in races:
    equal_odd_dict[race] = equalized_odds(race)

equal_odd_dict

{'AfricanAmerican': (0.015799379688784032, 0.07046142292315508),
 'Caucasian': (0.1444486643878972, 0.2693216707529874),
 'Other': (0.0007389493483810291, 0.004417644070417247),
 'Hispanic': (0.0011021886317115414, 0.006670642546330042),
 'Unknown': (0.0008129346942462289, 0.004991937799571488),
 'Asian': (0.0003329412025836237, 0.0013473814414772603)}

### Conditional Statistical Parity

In [85]:
L = ['gender', 'age']

In [86]:
import pandas as pd
import itertools

def get_combinations(df, columns):
    column_values = [df[column].unique() for column in columns]
    combinations = list(itertools.product(*column_values))
    return combinations

In [87]:
combinations = get_combinations(res, L)
combinations

[('Male', '30-60 years'),
 ('Male', 'Over 60 years'),
 ('Male', '30 years or younger'),
 ('Female', '30-60 years'),
 ('Female', 'Over 60 years'),
 ('Female', '30 years or younger')]

In [88]:
cond_stat_parity_dict = {}

In [89]:
def cond_stat_parity(race: str):
    for comb in res[L].drop_duplicates().values:
        len(res[res[L].isin(comb).all(axis=1)]), len(res) # all: 满足L中所有col的条件
        numerator = len(res[(res[L].isin(comb).all(axis=1)) & (res['predicted'] == 1) & (res['race'] == race)])
        denominator = len(res[(res[L].isin(comb).all(axis=1)) & (res['race'] == race)])
        cond_stat_parity_dict[(race,)+tuple(comb)] = numerator/denominator if denominator !=0 else 0

In [90]:
for race in races:
    cond_stat_parity(race)

In [91]:
len(cond_stat_parity_dict) # 6 group of race * 6 combinations of (age,gender)

36

In [92]:
cond_stat_parity_dict

{('AfricanAmerican', 'Male', '30-60 years'): 0.35823429541595925,
 ('AfricanAmerican', 'Female', '30-60 years'): 0.3650727650727651,
 ('AfricanAmerican', 'Male', 'Over 60 years'): 0.4205405405405405,
 ('AfricanAmerican', 'Female', 'Over 60 years'): 0.43197278911564624,
 ('AfricanAmerican', 'Male', '30 years or younger'): 0.2366412213740458,
 ('AfricanAmerican', 'Female', '30 years or younger'): 0.27710843373493976,
 ('Caucasian', 'Male', '30-60 years'): 0.3179135731962688,
 ('Caucasian', 'Female', '30-60 years'): 0.34221396117306896,
 ('Caucasian', 'Male', 'Over 60 years'): 0.3995428391266651,
 ('Caucasian', 'Female', 'Over 60 years'): 0.40480298116072044,
 ('Caucasian', 'Male', '30 years or younger'): 0.2049469964664311,
 ('Caucasian', 'Female', '30 years or younger'): 0.33190578158458245,
 ('Other', 'Male', '30-60 years'): 0.2883435582822086,
 ('Other', 'Female', '30-60 years'): 0.2288135593220339,
 ('Other', 'Male', 'Over 60 years'): 0.33495145631067963,
 ('Other', 'Female', 'Over 6


## Get Pairwise Ratio

In [93]:
# demographic parity
ratios = []
for i in range(len(races)):
    for j in range(i+1,len(races)):
        if demo_parity_dict[races[i]] < demo_parity_dict[races[j]]:
            temp = demo_parity_dict[races[i]]/demo_parity_dict[races[j]]
            ratios.append((races[i],races[j], temp))
        else:
            temp = demo_parity_dict[races[j]]/demo_parity_dict[races[i]]
            ratios.append((races[j],races[i], temp))    


In [94]:
len(ratios), ratios

(15,
 [('Caucasian', 'AfricanAmerican', 0.9689210129843422),
  ('Other', 'AfricanAmerican', 0.7792690825109174),
  ('Hispanic', 'AfricanAmerican', 0.8897942303685417),
  ('Unknown', 'AfricanAmerican', 0.5694565307125041),
  ('Asian', 'AfricanAmerican', 0.6763379251493356),
  ('Other', 'Caucasian', 0.8042648183578103),
  ('Hispanic', 'Caucasian', 0.9183351567821977),
  ('Unknown', 'Caucasian', 0.5877223458685652),
  ('Asian', 'Caucasian', 0.6980320543014844),
  ('Other', 'Hispanic', 0.8757857220406496),
  ('Unknown', 'Other', 0.730757248674146),
  ('Asian', 'Other', 0.8679132026771512),
  ('Unknown', 'Hispanic', 0.6399867646665255),
  ('Asian', 'Hispanic', 0.7601059908752216),
  ('Unknown', 'Asian', 0.8419704256371073)])

In [95]:
max_diff = float('-inf')
min_diff = float('inf')

for key1, value1 in demo_parity_dict.items():
    for key2, value2 in demo_parity_dict.items():
        if key1 != key2:
            diff = abs(value1 - value2)
            if diff > max_diff:
                max_diff = diff
                max_diff_keys = (key1, key2)
            if diff < min_diff:
                min_diff = diff
                min_diff_keys = (key1, key2)

print(max_diff_keys, min_diff_keys)
print(max_diff, min_diff)

('AfricanAmerican', 'Unknown') ('AfricanAmerican', 'Caucasian')
0.1691612261023723 0.012210984313125839


In [96]:
smallest_ratio = min(ratios, key=lambda x: x[2])
largest_ratio = max(ratios, key=lambda x: x[2])
maximum_parity = max(demo_parity_dict, key=demo_parity_dict.get)

In [97]:
print('Demographic Parity\n')
for race, score in demo_parity_dict.items():
    print(f"{race} | {score:.4f}")

print()
print(f'max difference: {max_diff_keys} | {max_diff:.4f}')
print(f'min difference: {min_diff_keys} | {min_diff:.4f}')
print(f'smallest ratio: {smallest_ratio}')
print(f'largest ratio: {largest_ratio}')
print(f'maximum indicator: {maximum_parity} | {demo_parity_dict[maximum_parity]:.4f}')

Demographic Parity

AfricanAmerican | 0.3929
Caucasian | 0.3807
Other | 0.3062
Hispanic | 0.3496
Unknown | 0.2237
Asian | 0.2657

max difference: ('AfricanAmerican', 'Unknown') | 0.1692
min difference: ('AfricanAmerican', 'Caucasian') | 0.0122
smallest ratio: ('Unknown', 'AfricanAmerican', 0.5694565307125041)
largest ratio: ('Caucasian', 'AfricanAmerican', 0.9689210129843422)
maximum indicator: AfricanAmerican | 0.3929


#### Fairness metrics

In [98]:
def fairness_metrics(data: dict):
    ratios = []
    for i in range(len(races)):
        for j in range(i+1,len(races)):
            if data[races[i]] < data[races[j]]:
                temp = data[races[i]]/data[races[j]]
                ratios.append((races[i],races[j], temp))
            else:
                temp = data[races[j]]/data[races[i]]
                ratios.append((races[j],races[i], temp)) 

    max_diff = float('-inf')
    min_diff = float('inf')

    for key1, value1 in data.items():
        for key2, value2 in data.items():
            if key1 != key2:
                diff = abs(value1 - value2)
                if diff > max_diff:
                    max_diff = diff
                    max_diff_keys = (key1, key2)
                if diff < min_diff:
                    min_diff = diff
                    min_diff_keys = (key1, key2)

    smallest_ratio = min(ratios, key=lambda x: x[2])
    largest_ratio = max(ratios, key=lambda x: x[2])
    maximum_parity = max(data, key=data.get)

    for race, score in data.items():
        print(f"{race} | {score:.4f}")

    print()
    print(f'max difference: {max_diff_keys} | {max_diff:.4f}')
    print(f'min difference: {min_diff_keys} | {min_diff:.4f}')
    print(f'smallest ratio: {smallest_ratio}')
    print(f'largest ratio: {largest_ratio}')
    print(f'maximum indicator: {maximum_parity} | {data[maximum_parity]:.4f}')


#### Demo Parity

In [99]:
print('Demographic Parity\n')
fairness_metrics(demo_parity_dict)

Demographic Parity

AfricanAmerican | 0.3929
Caucasian | 0.3807
Other | 0.3062
Hispanic | 0.3496
Unknown | 0.2237
Asian | 0.2657

max difference: ('AfricanAmerican', 'Unknown') | 0.1692
min difference: ('AfricanAmerican', 'Caucasian') | 0.0122
smallest ratio: ('Unknown', 'AfricanAmerican', 0.5694565307125041)
largest ratio: ('Caucasian', 'AfricanAmerican', 0.9689210129843422)
maximum indicator: AfricanAmerican | 0.3929


#### Equalized Opportunity

In [100]:
print('Equalized Opportunity\n')
fairness_metrics(equal_opp_dict)

Equalized Opportunity

AfricanAmerican | 0.0158
Caucasian | 0.1444
Other | 0.0007
Hispanic | 0.0011
Unknown | 0.0008
Asian | 0.0003

max difference: ('Caucasian', 'Asian') | 0.1441
min difference: ('Other', 'Unknown') | 0.0001
smallest ratio: ('Asian', 'Caucasian', 0.0023049102184119576)
largest ratio: ('Other', 'Unknown', 0.9089898039918192)
maximum indicator: Caucasian | 0.1444


#### Equalized Odds

In [101]:
equal_odd_true_dict = {}
equal_odd_false_dict = {}

for key, value in equal_odd_dict.items():
    equal_odd_true_dict[key] = value[0]
    equal_odd_false_dict[key] = value[1]

In [102]:
print('Equalized Odds (True)\n')
fairness_metrics(equal_odd_true_dict)
print('Equalized Odds (False)\n')
fairness_metrics(equal_odd_false_dict)

Equalized Odds (True)

AfricanAmerican | 0.0158
Caucasian | 0.1444
Other | 0.0007
Hispanic | 0.0011
Unknown | 0.0008
Asian | 0.0003

max difference: ('Caucasian', 'Asian') | 0.1441
min difference: ('Other', 'Unknown') | 0.0001
smallest ratio: ('Asian', 'Caucasian', 0.0023049102184119576)
largest ratio: ('Other', 'Unknown', 0.9089898039918192)
maximum indicator: Caucasian | 0.1444
Equalized Odds (False)

AfricanAmerican | 0.0705
Caucasian | 0.2693
Other | 0.0044
Hispanic | 0.0067
Unknown | 0.0050
Asian | 0.0013

max difference: ('Caucasian', 'Asian') | 0.2680
min difference: ('Other', 'Unknown') | 0.0006
smallest ratio: ('Asian', 'Caucasian', 0.005002870499466908)
largest ratio: ('Other', 'Unknown', 0.8849557522123895)
maximum indicator: Caucasian | 0.2693


#### Conditional Statistical Parity

In [103]:
data = cond_stat_parity_dict
max_diff_dict = {}
min_diff_dict = {}
max_diff_keys_dict = {}
min_diff_keys_dict = {}
ratios = {}

for comb in combinations:
    comb_ratios = []
    max_diff = float('-inf')
    min_diff = float('inf')
    max_diff_keys = None
    min_diff_keys = None
    for i in range(len(races)):
        for j in range(i+1,len(races)):
            id1 = (races[i],) + comb
            id2 = (races[j],) + comb
            diff = abs(data[id1]-data[id2])
            if diff > max_diff:
                max_diff = diff
                max_diff_keys = (races[i], races[j])
            if diff < min_diff:
                min_diff = diff
                min_diff_keys = (races[i], races[j])

            if data[id1] < data[id2]:
                temp = data[id1]/data[id2] if data[id2] != 0 else 0
                comb_ratios.append((races[i], races[j], comb, temp))
            else:
                temp = data[id2]/data[id1] if data[id1] != 0 else 0
                comb_ratios.append((races[j], races[i], comb, temp))

    ratios[comb] = comb_ratios
    max_diff_dict[comb] = max_diff
    min_diff_dict[comb] = min_diff
    max_diff_keys_dict[comb] = max_diff_keys
    min_diff_keys_dict[comb] = min_diff_keys

In [104]:
ratios[('Male', '30-60 years')]

[('Caucasian', 'AfricanAmerican', ('Male', '30-60 years'), 0.8874459460312906),
 ('Other', 'AfricanAmerican', ('Male', '30-60 years'), 0.8049021603233216),
 ('Hispanic', 'AfricanAmerican', ('Male', '30-60 years'), 0.812063765618268),
 ('Unknown', 'AfricanAmerican', ('Male', '30-60 years'), 0.4032122169562928),
 ('Asian', 'AfricanAmerican', ('Male', '30-60 years'), 0.48313889901567625),
 ('Other', 'Caucasian', ('Male', '30-60 years'), 0.9069872524888873),
 ('Hispanic', 'Caucasian', ('Male', '30-60 years'), 0.9150571584104519),
 ('Unknown', 'Caucasian', ('Male', '30-60 years'), 0.4543512974051896),
 ('Asian', 'Caucasian', ('Male', '30-60 years'), 0.5444150161216029),
 ('Other', 'Hispanic', ('Male', '30-60 years'), 0.991180981595092),
 ('Unknown', 'Other', ('Male', '30-60 years'), 0.5009456264775414),
 ('Asian', 'Other', ('Male', '30-60 years'), 0.6002454991816694),
 ('Unknown', 'Hispanic', ('Male', '30-60 years'), 0.49652777777777773),
 ('Asian', 'Hispanic', ('Male', '30-60 years'), 0.59

In [105]:
len(ratios), len(ratios[combinations[0]]) # 6 combinations of (gender, age), 6 * 15 = 90

(6, 15)

In [106]:
for comb in combinations:
    print(f'L = { comb }')
    max_val = -1
    for race in races:
        max_val = max(max_val,data[(race,) + comb])
        print(f"{race} | {data[(race,) + comb]:.4f}")
    print(f'Maximum indicator: {max_val:.4f}')
    print()

    print(f'max difference: {max_diff_keys_dict[comb]} | {max_diff_dict[comb]:.4f}')
    print(f'min difference: {min_diff_keys_dict[comb]} | {min_diff_dict[comb]:.4f}')
    temp = min(ratios[comb], key=lambda x: x[3])
    print(f'smallest ratio: {temp[0]}, {temp[1]} | {temp[3]}')
    temp = max(ratios[comb], key=lambda x: x[3])
    print(f'largest ratio: {temp[0]}, {temp[1]} | {temp[3]}')
    print('---------------------')


L = ('Male', '30-60 years')
AfricanAmerican | 0.3582
Caucasian | 0.3179
Other | 0.2883
Hispanic | 0.2909
Unknown | 0.1444
Asian | 0.1731
Maximum indicator: 0.3582

max difference: ('AfricanAmerican', 'Unknown') | 0.2138
min difference: ('Other', 'Hispanic') | 0.0026
smallest ratio: Unknown, AfricanAmerican | 0.4032122169562928
largest ratio: Other, Hispanic | 0.991180981595092
---------------------
L = ('Male', 'Over 60 years')
AfricanAmerican | 0.4205
Caucasian | 0.3995
Other | 0.3350
Hispanic | 0.4741
Unknown | 0.2959
Asian | 0.3131
Maximum indicator: 0.4741

max difference: ('Hispanic', 'Unknown') | 0.1782
min difference: ('Unknown', 'Asian') | 0.0172
smallest ratio: Unknown, Hispanic | 0.6241701019743979
largest ratio: Caucasian, AfricanAmerican | 0.950069733141813
---------------------
L = ('Male', '30 years or younger')
AfricanAmerican | 0.2366
Caucasian | 0.2049
Other | 0.0833
Hispanic | 0.2941
Unknown | 0.0000
Asian | 0.0000
Maximum indicator: 0.2941

max difference: ('Hispanic